# Линейные модели и нейронные сети. Практическая работа

В этой практической работе пять обязательных задач.

Они помогут понять, что вы действительно усвоили материал модуля. 

Удачи!

## Цели практической работы
Потренироваться в обучении модели:
- линейной регрессии;
- логистической регрессии; 
- многослойного персептрона.

## Что входит в практическую работу

1. Загрузите датасет и ознакомьтесь с ним.
2. Обучите линейную регрессию, включая признаки с нулевым весом и без них. Замерьте качество.
3. Обучите случайный лес. Замерьте качество.
4. Обучите многослойный перцептрон. Замерьте качество.
5. Реализуйте стратегию голосования. Замерьте качество.

## Что оценивается

* Выполнены все задания. В каждом:
 * в коде нет ручных перечислений, все действия автоматизированы;
 * результаты вычислений и применённых операций корректны;
 * ответы на вопросы, где требуется, корректны и обоснованы; 
 * код читабелен: переменным даны осмысленные названия, соблюдены отступы и правила расстановки пробелов. Стилизация кода соответствует рекомендациям [PEP 8](https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html).
 
* Репозиторий проекта оформлен корректно:
 * содержит осмысленные коммиты, содержащие конкретные реализованные фичи;
 * ветки названы согласно назначению;
 * файлы, не связанные с проектом, не хранятся в репозитории.

## Как отправить работу на проверку

Сдайте практическую работу этого модуля через систему контроля версий Git сервиса Skillbox GitLab. После загрузки работы на проверку напишите об этом в личном кабинете своему куратору.

## Задача

Постройте модель классификации, определяющую категорию цены / цену подержанного автомобиля в зависимости от характеристик транспортного средства. 

Используйте датасет из коллекции подержанных автомобилей, выставленных на продажу в Соединенных Штатах. Он уже подготовлен, без выбросов и с категориальными фичами, преобразованными с помощью one hot encoding, и количественными фичами, стандартизированными с помощью скейлеров.

### Описание датасета:
- `id`— идентификатор записи;
- `is_manufacturer_name`— признак производителя автомобиля;

- `region_*`— регион;
- `x0_*`— тип топлива;
- `manufacturer_*`— производитель;
- `short_model_*`— сокращённая модель автомобиля;
- `title_status_*`— статус;
- `transmission_*`— коробка передач;
- `state_*`— штат;
- `age_category_*`— возрастная категория автомобиля;

- `std_scaled_odometer`— количество пройденных миль (после стандартизации);
- `year_std`— год выпуска (после стандартизации);
- `lat_std`— широта (после стандартизации);
- `long_std`— долгота (после стандартизации);
- `odometer/price_std`— отношение стоимости к пробегу автомобиля (после стандартизации);
- `desc_len_std`— количество символов в тексте объявления о продаже (после стандартизации);
- `model_in_desc_std`— количество наименований модели автомобиля в тексте объявления о продаже (после стандартизации);
- `model_len_std`— длина наименования автомобиля (после стандартизации);
- `model_word_count_std`— количество слов в наименовании автомобиля (после стандартизации);
- `month_std`— номер месяца размещения объявления о продаже автомобиля (после стандартизации);
- `dayofweek_std`— день недели размещения объявления о продаже автомобиля (после стандартизации);
- `diff_years_std`— количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля (после стандартизации);
- `price`— стоимость;
- `price_category`— категория цены.

## Обязательные задачи

In [39]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error
from sklearn.model_selection import train_test_split 
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sympy.physics.quantum.matrixutils import sparse

In [2]:
df = pd.read_csv(r'.\data\vehicles_dataset_prepared.csv')
df.head()

,id,price,price_category,is_audi,is_ford,is_chevrolet,is_toyota,x0_diesel,x0_electric,x0_gas,...,long_std,year_std,odometer/price_std,desc_len_std,model_in_desc_std,model_len_std,model_word_count_std,month_std,dayofweek_std,diff_years_std
0,7308295377,54990,high,0,0,0,0,1.0,0.0,0.0,...,0.484245,1.322394,-0.510784,0.632075,-0.155788,1.163032,1.910669,-0.615846,1.120284,-1.322394
1,7316380095,16942,medium,0,1,0,0,0.0,0.0,0.0,...,1.110800,0.695973,-0.402947,-0.646781,-0.155788,0.932087,1.235799,1.623784,-1.374972,-0.695973
2,7313733749,35590,high,0,0,0,0,0.0,0.0,1.0,...,0.531185,0.852578,-0.514480,0.560744,-0.155788,0.470197,0.560930,-0.615846,-0.376870,-0.852578
3,7308210929,14500,medium,0,0,0,1,0.0,0.0,1.0,...,0.853562,0.226157,-0.241883,0.180435,-0.155788,-0.915473,-0.788810,-0.615846,1.120284,-0.226157
4,7303797340,14590,medium,0,0,0,0,0.0,0.0,0.0,...,0.557607,0.069552,-0.333074,0.766366,-0.155788,1.163032,1.910669,-0.615846,0.122182,-0.069552


**Задача 1. Линейная регрессия**

Вспомните задачу по предсказанию стоимости подержанного автомобиля. Попробуйте обучить модель линейной регрессии для предсказания цены автомобиля (колонка `price`). Для этого сделайте шаги:

- подготовьте данные: удалите колонки, которые косвенно содержат информацию о целевой переменной (`odometer/price_std`, `price_category`);
- разделите выборку на треин и тест в отношении 70/30;
- обучите модель линейной регрессии с дефолтными параметрами;
- посчитайте значение метрики mae на тестовой выборке для линейной регрессии;
- выведите получившиеся коэффициенты линейной регрессии при каждом параметре обучающей выборки с помощью метода `coef_` (есть ли коэффициенты, которые равны нулю? Если есть, выведите названия фичей с нулевым коэффициентом);
- удалите фичи, коэффициенты которых равны нулю; переобучите модель; убедитесь, что значение метрики не изменилось.



In [3]:
df_prepared = df.copy()
df_prepared.drop(['odometer/price_std', 'price_category'], axis=1, inplace=True)
df_prepared.head()

x = df_prepared.drop(['price'], axis=1)
y = df_prepared['price']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [4]:
model = LinearRegression()
model.fit(x_train, y_train);

In [5]:
mae = mean_absolute_error(y_test, model.predict(x_test))
mae

4600.341117047692

In [6]:
features_weights = [[feature, weight] for feature, weight in list(zip(x.columns, model.coef_))]
features_weights = pd.DataFrame(features_weights, columns=['feature', 'weight'])
features_weights[features_weights['weight'] == 0]

,feature,weight
1360,manufacturer_ferrari,0.0


**Задача 2. Логистическая регрессия**

Теперь в рамках тех же данных попробуйте предсказать `price_category` с помощью алгоритма логистической регрессии. Предварительно из датафрейма удалите переменные, в которых косвенно содержится информация о целевой переменной (`odometer/price_std`, `price`). 

Для обученной модели:

- рассчитайте и выведите метрику качества (accuracy) на тренировочной выборке;
- сделайте предикт на тестовых данных и положите его в переменную `logreg_pred`;
- рассчитайте и выведите accuracy и confusion_matrix на тестовой выборке.

Обратите внимание, что это задание засчитывается, если: 
- accuracy на тренировочной выборке > 87%;
- accuracy на тестовой выборке > 75.5%.

Для улучшения метрики можно преобразовывать входные данные и тюнить параметры логистической регрессии.

In [7]:
# Ваш код подготовки данных здесь. Допишите инициализацию входных фичей и целевую переменную
df_prepared = df.copy()
df_prepared.drop(['odometer/price_std', 'price'], axis=1, inplace=True)

x = df_prepared.drop(['price_category', 'id'], axis=1)
y = df_prepared['price_category']


In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [9]:
from sklearn.model_selection import GridSearchCV

model2= LogisticRegression()

param_grid = {
    'solver': ['newton-cg', 'sag', 'saga', 'lbfgs'],
    'C': [0.5 * i for i in range(5)],
    'class_weight': [None, 'balanced'],
    'max_iter': list(range(1000, 5000, 500)),
}
randomized_search_rf = GridSearchCV(model2, param_grid, scoring='accuracy', n_jobs=-1)

randomized_search_rf.fit(x_train, y_train)

best_params = randomized_search_rf.best_params_
print(best_params)

C:\Users\kuza-\miniconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
320 fits failed out of a total of 1600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
320 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\kuza-\miniconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\kuza-\miniconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\kuza-\miniconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\kuza-\miniconda3\Lib\site-

{'C': 1.0, 'class_weight': 'balanced', 'max_iter': 1000, 'solver': 'sag'}


In [25]:
best_params = {'C': 1.0, 'class_weight': 'balanced', 'max_iter': 1000, 'solver': 'sag'}

In [29]:
model2= LogisticRegression(**best_params)
model2.fit(x_train, y_train);

In [34]:
y_pred = model2.predict(x_test)
print('Accuracy in training set', round(model2.score(x_train, y_train), 3))
print('Accuracy in test set', round(accuracy_score(y_test, y_pred), 3))
confusion_matrix(y_test, y_pred)

Accuracy in training set 0.855
Accuracy in test set 0.769


array([[543,  18,  90],
       [ 15, 489,  87],
       [ 94, 140, 448]], dtype=int64)

In [31]:
model2.coef_

array([[ 0.34055647,  0.31761501,  0.27677582, ..., -0.03831442,
         0.02858717, -0.53196449],
       [-0.34388077, -0.30193261, -0.19807822, ...,  0.03663543,
        -0.02273028,  0.46127481],
       [ 0.0033243 , -0.0156824 , -0.0786976 , ...,  0.00167899,
        -0.00585689,  0.07068967]])

**Задача 3. Многослойный персептрон**

Решите задачу, поставленную в предыдущем задании, применив модель многослойного персептрона. С помощью данного алгоритма добейтесь лучших значений точности на тренировочной и тестовой выборках, чем на логистической регрессии. 

Увеличение точности в данном задании, по сравнению с предыдущим, должно быть больше увеличения 0.01 по метрике accuracy.

In [37]:
from sklearn.neural_network import MLPClassifier

model3 = MLPClassifier(hidden_layer_sizes=(100, 100))
model3.fit(x_train, y_train)

MLPClassifier(hidden_layer_sizes=(100, 100))

In [38]:
y_pred = model3.predict(x_test)
print('Accuracy in training set', round(model3.score(x_train, y_train), 3))
print('Accuracy in test set', round(accuracy_score(y_test, y_pred), 3))
confusion_matrix(y_test, y_pred)

Accuracy in training set 0.991
Accuracy in test set 0.778


array([[531,  19, 101],
       [ 12, 500,  79],
       [ 73, 143, 466]], dtype=int64)

**Задача 4. Сравнение с древовидными моделями**

Обучите модель случайного леса на тех же данных для предсказания `price_category`. Сравните качество с моделью логистической регрессии и многослойного персептрона. Словами опишите, какая из моделей в каких случаях работает лучше по результатам на тестовой выборке, обоснуйте свой выбор.

In [40]:
model4 = RandomForestClassifier()
model4.fit(x_train, y_train)

RandomForestClassifier()

In [42]:
y_pred = model4.predict(x_test)
print('Accuracy in training set', round(model4.score(x_train, y_train), 3))
print('Accuracy in test set', round(accuracy_score(y_test, y_pred), 3))
confusion_matrix(y_test, y_pred)

# Персептрон справился лучше всех на тестовой выборке. Деревья по понятным причинам показала идеальную точность на обучающей выборке, но хуже всех на тестовой, однако она обучилась гораздо быстрее всех остальных моделей. Линейная регрессия обучалась дольше всех.

Accuracy in training set 1.0
Accuracy in test set 0.753


array([[529,  26,  96],
       [ 15, 500,  76],
       [ 98, 164, 420]], dtype=int64)

**Задача 5. Стратегия голосования**

Реализуйте стратегию голосования для предсказания целевой переменной.
Голосование в задаче классификации — это когда несколько моделей выдают свои предикты, и финальным выбирается тот предикт, который предсказали большинство моделей.

Для реализации этой стратегии проделайте следующее:

- сохраните предсказания каждой из моделей (случайный лес, многослойный персептрон, логистическая регрессия) для тестовой выборки в датафрейм `pred_df`;
- в четвёртую колонку `target` положите тот класс, который предсказало большинство классификаторов; например, если в строке были значения `high, medium, medium`, в `target` нужно положить `medium`;

     если в строке три разных класса (`high, medium, low`), придумайте свою стратегию по выбору значения; самая простая стратегия — выбрать рандомно одно значение из трёх;

- посчитайте точность предсказания с помощью голосования; выведите значения метрик accuracy и confusion_matrix.

Добейтесь значения точности > 78%.

In [60]:
import numpy as np

lr_predicts = model2.predict(x_test)
MLP_predicts = model3.predict(x_test)
rf_predicts = model4.predict(x_test)

pred_df = []
for p1, p2, p3 in zip(lr_predicts, MLP_predicts, rf_predicts):
    # get most popular answer
    unique, pos = np.unique([p1, p2, p3], return_inverse=True)
    counts = np.bincount(pos)
    maxpos = counts.argmax()
    pop_answer = unique[maxpos]
    
    # creating df
    pred_df.append([p1, p2, p3, pop_answer])


pred_df = pd.DataFrame(pred_df, columns=['lr_predict', 'MLP_predict', 'rf_predict', 'target'])
pred_df.head()

,lr_predict,MLP_predict,rf_predict,target
0,high,high,high,high
1,low,medium,medium,medium
2,medium,low,low,low
3,high,high,high,high
4,high,low,low,low


In [61]:
print('Accuracy in test set', round(accuracy_score(y_test, pred_df.target), 3))
confusion_matrix(y_test, y_pred)

Accuracy in test set 0.796


array([[529,  26,  96],
       [ 15, 500,  76],
       [ 98, 164, 420]], dtype=int64)

**Примечание**

В этой практической работе перед вами встал выбор: включать ли колонку `id`. При удалении данной колонки во время обучения логистической регрессии качество заметно улучшается.

Обучать любую модель (будь то логистическая, линейная регрессия или древовидный алгоритм) на данных айдишников не считается хорошей практикой. Как правило, между `id` и целевой переменной не должно быть никаких взаимосвязей. Включая колонку `id` в качества атрибута в обучение, вы стараетесь подогнать результаты своей модели под айдишники записей. Таким образом модель обучится на некотором наборе частных случаев и, возможно, не обратит внимание на общие зависимости.   